# 계층(layer)에 대한 이해
- 하나의 계층은 하나의 일만 전문적으로 할 수 있어야 한다. (국소적 계산)
- Fully Connected 계층은 WX+B 계산만
- ReLU 계층은 ReLU 연산만
- 덧셈 계층은 덧셈 연산만
- 곱셈 계층은 곱셈 연산만
- 모든 레이어는 미분값을 역전파한다.
  - 순전파 : 계산
  - 역전파 : 미분값을 넘겨 주는 것

In [31]:
# 곱셈 계층 구현하기
# forward : x*y
# backward : dx = 미분값 * y, dy = 미분값 * x

# 비고 : forward할 때 들어온 각 값들은 저장하고 있어야 한다.
# 이제 알고리즘만 알면 forwarding, backwarding 할 수 있음. 레이어를 마음대로 구성 가능

In [32]:
class MulLayer:
  
  # 초기화 : 객체에서 사용할 변수를 미리 준비
  def __init__(self):
    self.x = None
    self.y = None
  
  # 순전파 : x * y 해서 리턴
  # 계산되는 값들 각각 저장하고 있어야 한다.
  def forward(self, x, y):
    self.x = x
    self.y = y

    out = x * y
    return out
  
  # dout : 뒷층에서 넘어온 미분값
  # dx = dout * y
  # dy = dout * x
  
  # return dx, dy
  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x

    return dx, dy    

In [33]:
# 입력값 3개 : 사과의 가격, 사과의 갯수, 소비세
# 레이어 3개
# apple layer, tax layer

apple = 100  # 사과 1개 당 가격
apple_cnt = 2  # 사과 갯수
tax = 1.1  # 소비세

# 계층은 2개
# (apple * apple_cnt) * tax
#       1개          2개

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파 먼저 수행
apple_price = mul_apple_layer.forward(apple, apple_cnt)
price       = mul_tax_layer.forward(apple_price, tax)

print("최종 사과 가격 : {:.0f}".format(price))

최종 사과 가격 : 220


In [34]:
# d돈통 / d포스기
dprice = 1  # 포스기에 찍힌 금액 그대로 돈통에 들어가야 함
dapple_price, dtax = mul_tax_layer.backward(dprice)  # backward 함수 - x, y가 들어갔으니까 dx, dy가 나옴
dapple, dapple_cnt = mul_apple_layer.backward(dapple_price)

print("사과 가격 * 사과 개수에 대한 미분값 : {}".format(dapple_price))
print("사과 가격에 대한 미분값 : {}".format(dapple))
print("사과 개수에 대한 미분값 : {}".format(dapple_cnt))
print("소비세에 대한 미분값 : {}".format(dtax))

사과 가격 * 사과 개수에 대한 미분값 : 1.1
사과 가격에 대한 미분값 : 2.2
사과 개수에 대한 미분값 : 110.00000000000001
소비세에 대한 미분값 : 200


In [35]:
# 각각이 미분값의 의미?
# 3. 사과 개수에 대한 미분값?
# 갯수만 바뀌는 거지 나머지 가격, tax는 신경을 안 쓰겠다는 얘기
# 국소적 미분이니까 갯수만 신경씀
# 갯수가 변경되면 가격이 110배 변경된다는 이야기

In [36]:
# 덧셈 계층 구현하기
# forward : x+ y
# backward : 뒷층에서 보내진 미분값에 * 1 을 곱한다
# 비고 : forward 시에 입력된 값을 가지고 있지 않아도 된다. 역전파 시에는 미분값만 필요하니까.

In [37]:
class AddLayer:

  def __init__(self):
    pass
  
  def forward(self, x, y):
    out = x + y
    return out
  
  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

In [38]:
# 입력값 : 사과의 갯수, 사과의 가격, 귤의 갯수, 귤의 가격, 소비세
apple = 100
apple_cnt = 2

orange = 150
orange_cnt = 3

tax = 1.1

# 1계층
mul_apple_layer = MulLayer()  # 사과 갯수 * 사과 가격
mul_orange_layer = MulLayer()  # 귤 갯수 * 귤 가격

# 2계층
add_apple_orange_layer = AddLayer()  # 사과 총 가격 + 귤 총 가격

# 3계층
mul_tax_layer = MulLayer()  # 과일들의 총 가격 * tax

# 1계층 연산
apple_price = mul_apple_layer.forward(apple, apple_cnt)
orange_price = mul_orange_layer.forward(orange, orange_cnt)

# 2계층 연산
total_fruit_price = add_apple_orange_layer.forward(apple_price, orange_price)

# 3계층 연산
final_money = mul_tax_layer.forward(total_fruit_price, tax)

print("최종 가격 : {}".format(int(final_money)))

최종 가격 : 715


In [39]:
# 국소적 계산이기 때문에
# 3계층 역전파 -> 2계층 역전파를 순서대로 한 다음
# 1계층 역전파를 할 때 사과부터 하나 귤부터 하나는 순간 상관 없음(국소적 계산)

# 역전파
dprice = 1  # d돈통 / d포스기

# dprice / dtotal_price, dprice / dtax
# 이 2개를 구하는 것이 목적
# 세금이 적용되기 전, 세금이 적용될 때의 가격을 구하겠다.
dtotal_price, dtax = mul_tax_layer.backward(dprice)

# d돈통 / d포스기 = d돈통 / dprice * dprice / dtotal_price * dtotal_price / dapple_price
dapple_price, dorange_price = add_apple_orange_layer.backward(dtotal_price)

In [40]:
# 미분값이 곱해진다 -> 미분값이 뒤로 전달된다.

# 미분의 정확한 의미 파악!

In [41]:
dorange, dorange_cnt = mul_orange_layer.backward(dorange_price)
dapple, dapple_cnt = mul_apple_layer.backward(dapple_price)

print("사과 2개 오렌지 3개의 가격 : {}".format(price))
print("사과 개수 미분 : {}".format(dapple_cnt))
print("사과 가격 미분 : {}".format(dapple))
print("오렌지 개수 미분 : {}".format(dorange_cnt))
print("오렌지 가격 미분 : {}".format(dorange))
print("소비세 미분 : {}".format(dtax))

사과 2개 오렌지 3개의 가격 : 220.00000000000003
사과 개수 미분 : 110.00000000000001
사과 가격 미분 : 2.2
오렌지 개수 미분 : 165.0
오렌지 가격 미분 : 3.3000000000000003
소비세 미분 : 650
